In [6]:
# imports
import pandas as pd
import json
import pyarrow

In [7]:
# reading in the given csv files
df_movies = pd.read_csv("data/movies_groupA.csv")
df_credits = pd.read_csv("data/credits_groupA.csv")
df_recom = pd.read_csv("C:/Users/ibele/OneDrive/DHBW/4. Semester/Big Data Storage/Prüfungsleistung/groupA/recommendations_groupA.csv")

In [8]:
# method for normalizing the dataframes with json
def get_normalized_df(df: pd.DataFrame, col: str, key_attribute: str="movie_id") -> pd.DataFrame:
    df[col] = df[col].map(json.loads).map(pd.json_normalize)
    clean_dfs = [row[col].assign(id=row[key_attribute]) for _, row in df.iterrows()]
    # creating the cleaned dataframe 
    df_clean = pd.concat(clean_dfs, ignore_index=True)

    return df_clean

# creating a schema for the cleaned dataframes
new_tables = [
    ("production_companies", df_movies, "id"),
    ("spoken_languages", df_movies, "id"),
    ("genres", df_movies, "id"),
    ("cast", df_credits, "movie_id"),
    ("crew", df_credits, "movie_id")
]

# saving the cleaned dataframes as parquet
for table, df, id_name in new_tables:
    normalized_df = get_normalized_df(df, table, id_name)
    normalized_df.to_parquet(f"filtered_data/{table}.parquet", index=False)

In [9]:
# saving movies and recoms to the clean / filtered data as parquet
df_recom.to_parquet("filtered_data/recom.parquet")
df_movies[["title","release_date","runtime","revenue","budget","popularity","id"]].to_parquet("filtered_data/movies.parquet")